<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/selenium_streetview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title setup
# https://colab.research.google.com/github/kaliiiiiiiiii/Selenium-Profiles/blob/master/google-colab/selenium_profiles.ipynb
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay 


In [107]:
#@title Start actual driver
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
# from selenium.webdriver.common.by import By  # locate elements
from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display
from webdriver_manager.chrome import ChromeDriverManager

chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()

profile = profiles.Windows() # or .Android
profile["cdp"]["cores"] = None # Chrome 90 doesn't allow emulating cores :(driver = mydriver.start(profile, uc_driver=False, executable_path=chromedriver_path)

mydriver = Chrome(profile, executable_path=chromedriver_path)
display = display()
display.start_display()
driver = mydriver.start()


In [ ]:
# @title by meta

import requests

# olat, olng = 1.39001828832101, 103.850718308561 # both good, meta slightly better
# olat, olng = 1.39197802872949,103.903760849909 # og urlchange better
# olat, olng = 1.41776879691454, 103.759493238762 # tree
olat, olng = 1.30432686507431, 103.838930405718
# location = str(olat)+', '+str(olng)
# location = "60 ST ANNE'S WOOD"
location = "7 THIRD ST"

api_key = '_'
uuu='https://maps.googleapis.com/maps/api/streetview/metadata?'+'location='+location+'&key='+api_key
# pano = 'ItO6rlhUWJeSqpw_gcd4Ew'
# # uu = 'https://www.google.com/maps/@?api=1&map_action=pano&pano='+pano+'&heading='+str(heading)+'&pitch=5&fov=80'
# uuu='https://maps.googleapis.com/maps/api/streetview/metadata?'+'pano='+pano+'&key='+api_key

response = requests.get(uuu)
print(response.json())

lat, lng = response.json()['location'].values()
print(lat, lng)
# 1.3919781%2C103.9036102
# 1.391995753586262 103.9036096633161 onemap->gmap meta
# 1.391998996287508 103.9031654912029 name in gmap

# lat, lng = 1.3919781,103.9036102

# import math
# heading = 180-math.copysign(1, olng-lng)*(90)-math.atan((olat-lat)/(olng-lng))*(180/math.pi)
# print(heading)

# # https://developers.google.com/maps/documentation/urls/get-started#forming-the-street-view-url
# # heading -180 to 360

# # https://www.google.com/maps/@?api=1&map_action=pano&viewpoint=1.33710301745449%2C103.954313017682&heading=-45&pitch=38&fov=80
# url = 'https://www.google.com/maps/@?api=1&map_action=pano&viewpoint='+str(lat)+'%2C'+str(lng)+'&heading='+str(heading)+'&pitch=5&fov=80'
# print(url)



{'copyright': '© Google', 'date': '2023-01', 'location': {'lat': 37.75283032075294, 'lng': -122.3879088752893}, 'pano_id': 'BhvduL0Txkp1P7M2nbfwBg', 'status': 'OK'}
37.75283032075294 -122.3879088752893


In [ ]:
# @title by change url
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import math

uu = 'https://www.google.com/maps/@?api=1&map_action=pano&viewpoint='+str(olat)+'%2C'+str(olng)#+'&heading='+str(heading)+'&pitch=5&fov=80'
print(uu)
driver.get(uu)

WebDriverWait(driver, 7).until(lambda driver: "/data=" in driver.current_url)
get_url = driver.current_url
# https://www.google.com/maps/@1.3919781,103.9036102,3a,90y,90t/data=!3m6!1e1!3m4!1sItO6rlhUWJeSqpw_gcd4Ew!2e0!7i16384!8i8192
print(get_url)
gllayt,dat = get_url.split('/data=')

# # lat,lng,?,fov,pitch
lati, lngi, a, y, t = gllayt.split('@')[1].split(',')
lat, lng = float(lati), float(lngi)
heading = 180-math.copysign(1, olng-lng)*(90)-math.atan((olat-lat)/(olng-lng))*(180/math.pi)
# # print(heading)
fov='80'
pitch='95' # 0 floor 90 level 180 up
# url = 'https://www.google.com/maps/@'+lati+','+lngi+','+a+','+fov+'y,'+str(heading)+'h,'+pitch+'t,'+'/data='+dat
url = 'https://www.google.com/maps/@?api=1&map_action=pano&viewpoint='+str(olat)+'%2C'+str(olng)+'&heading='+str(heading)+'&pitch=5&fov=80'
print(url)

# https://www.trekview.org/blog/2020/decoding-google-street-view-urls/


In [ ]:

# @title by address name


# https://maps.google.com/maps
location = "7 THIRD ST"
location = location.replace(' ','+')
location = location+'+singapore'
print(location)
uuu = 'https://www.google.com/maps/search/?api=1&query='+location
print(uuu)

driver.get(uuu)

WebDriverWait(driver, 7).until(lambda driver: "/data=" in driver.current_url)
get_url = driver.current_url
# https://www.google.com/maps/@1.3919781,103.9036102,3a,90y,90t/data=!3m6!1e1!3m4!1sItO6rlhUWJeSqpw_gcd4Ew!2e0!7i16384!8i8192
print(get_url)
gllayt,dat = get_url.split('/data=')

# # lat,lng,?
olati, olngi, z = gllayt.split('@')[1].split(',')
# olati, olngi = gllayt.split('@')[1].split(',')[:2]
olat, olng = float(olati), float(olngi)




driver.get(uuu)

WebDriverWait(driver, 7).until(lambda driver: "/data=" in driver.current_url)
get_url = driver.current_url
# https://www.google.com/maps/@1.3919781,103.9036102,3a,90y,90t/data=!3m6!1e1!3m4!1sItO6rlhUWJeSqpw_gcd4Ew!2e0!7i16384!8i8192
print(get_url)
gllayt,dat = get_url.split('/data=')

# # lat,lng,?,fov,pitch
# lati, lngi, a, y, t = gllayt.split('@')[1].split(',')
lati, lngi = gllayt.split('@')[1].split(',')[:2]
lat, lng = float(lati), float(lngi)
heading = 180-math.copysign(1, olng-lng)*(90)-math.atan((olat-lat)/(olng-lng))*(180/math.pi)
# # print(heading)
fov='80'
pitch='95' # 0 floor 90 level 180 up
# url = 'https://www.google.com/maps/@'+lati+','+lngi+','+a+','+fov+'y,'+str(heading)+'h,'+pitch+'t,'+'/data='+dat
url = 'https://www.google.com/maps/@?api=1&map_action=pano&viewpoint='+str(olat)+'%2C'+str(olng)+'&heading='+str(heading)+'&pitch=5&fov=80'
print(url)


In [ ]:
#@title Get screenshot

# 1.31525700666637,103.80869896485

# url="https://www.google.com/maps/@?api=1&map_action=pano&viewpoint=48.857832%2C2.295226&heading=-45&pitch=38&fov=80"

driver.get(url)
# import time
# time.sleep(2) # black 1.2 blur 1.3 1.5 done 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, "*//']")))

WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, "*//div[@role='contentinfo']/span[7]/span/span/span")))
element = driver.find_elements(By.XPATH,"*//div[@role='contentinfo']/span[7]/span/span/span")[0]
mon, yyyy = element.text.split(': ')[1].split(' ')
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
mm = months.index(mon)+1
date= yyyy+'-'+str(mm)
print(date)

delist=["*//div[@id='titlecard']",
"*//div[@id='image-header']",
"*//div[contains(@class, 'scene-footer-container')]",
"*//div[contains(@class, 'app-viewcard-strip')]",
"*//div[@id='watermark']",]
for delete in delist:
    delement = driver.find_element(By.XPATH, delete)
    driver.execute_script("arguments[0].remove();", delement)
# print(driver.page_source) # https://www.geeksforgeeks.org/web-driver-methods-in-selenium-python/

element = driver.find_elements(By.XPATH,"*//div[@aria-label='Street View']")[0]

scrrenshot = element.screenshot_as_png
with open('canvas.png', 'wb') as f:
    f.write(scrrenshot)

# showscreen(driver)
# # show_html(driver)


In [104]:
# @title as funcs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import requests
import math
from PIL import Image
import matplotlib.pyplot as plt

# https://maps.google.com/maps

def gmap_latlng(location):
    location = location.replace(' ','+')
    location = location+'+singapore'
    uuu = 'https://www.google.com/maps/search/?api=1&query='+location
    driver.get(uuu)
    WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "*//button[@aria-label='Search']")))
    driver.find_elements(By.XPATH,"*//button[@aria-label='Search']")[0].click()
    # print(driver.current_url)
    # for x in range(4):
    #     time.sleep(1)
    #     driver.find_elements(By.XPATH,"*//button[@aria-label='Search']")[0].click()
    #     print(driver.current_url)
    try: WebDriverWait(driver, 3).until(lambda driver: "/data=" in driver.current_url)
    except: return
    if not '!16s' in driver.current_url: return
    olat = driver.current_url.split('!3d')[1].split('!4d')[0]
    olng = driver.current_url.split('!4d')[1].split('!16s')[0]
    return float(olat), float(olng)


def heading_url(olat, olng):
    api_key = '_'
    location = str(olat)+', '+str(olng)
    uuu='https://maps.googleapis.com/maps/api/streetview/metadata?'+'location='+location+'&key='+api_key
    response = requests.get(uuu)
    # print(response.json())
    if response.json()['status'] != 'OK': return
    lat, lng = response.json()['location'].values()
    dist = ((olat-lat)**2 + (olng-lng)**2)**(1/2)*111111
    print('dist: ',dist)
    heading = 180-math.copysign(1, olng-lng)*(90)-math.atan((olat-lat)/(olng-lng))*(180/math.pi)
    url = 'https://www.google.com/maps/@?api=1&map_action=pano&viewpoint='+str(lat)+'%2C'+str(lng)+'&heading='+str(heading)+'&pitch=5&fov=80'
    return url


def gmap_direct_old(location):
    location = location.replace(' ','+')
    location = location+'+singapore'
    uuu = 'https://www.google.com/maps/search/?api=1&query='+location
    driver.get(uuu)
    WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "*//button[@aria-label='Search']")))
    driver.find_elements(By.XPATH,"*//button[@aria-label='Search']")[0].click()
    # if not '!16s' in driver.current_url: return
    try: WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//button[@jsaction='pane.heroHeaderImage.click']")))
    except: return
    driver.find_elements(By.XPATH,"*//button[@jsaction='pane.heroHeaderImage.click']")[0].click()

    # try: WebDriverWait(driver, 7).until(lambda driver: "/data=" in driver.current_url)
    # except: return
    # uu = driver.current_url
    # fovl = uu.find(',75y,',33)
    # pitchl = uu.find(',90t/data',fovl)
    # fov, pitch = 80, 95
    # url = uu[:fovl+1] + str(fov) + uu[fovl+3:pitchl+1] + str(pitch) + uu[pitchl+3:]
    # driver.get(url)

    WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, "*//div[@role='contentinfo']/span[7]/span/span/span")))
    element = driver.find_elements(By.XPATH,"*//div[@role='contentinfo']/span[7]/span/span/span")[0]
    mon, yyyy = element.text.split(': ')[1].split(' ')
    months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    mm = months.index(mon)+1
    date= yyyy+'-'+str(mm).zfill(2)
    # time.sleep(1)

    # WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "*//button[contains(@jsaction, 'drawer.close')]")))
    driver.find_elements(By.XPATH,"*//button[contains(@jsaction, 'drawer.close')]")[0].click()
    # print(driver.page_source)
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//button[contains(@jsaction, 'drawer.open')]")))
    delist=["*//div[@id='titlecard']",
    "*//div[@id='image-header']",
    "*//div[contains(@class, 'scene-footer-container')]",
    "*//div[contains(@class, 'app-viewcard-strip')]",
    "*//div[@id='watermark']",
    "*//button[contains(@jsaction, 'drawer.open')]",]
    # time.sleep(1)
    for delete in delist:
        delement = driver.find_element(By.XPATH, delete)
        driver.execute_script("arguments[0].remove();", delement)
    element = driver.find_elements(By.XPATH,"*//div[@aria-label='Google Maps']")[0]
    # element = driver.find_elements(By.XPATH,"*//div[@id='content-container']")[0]
    # element = driver.find_elements(By.XPATH,"*//div[@class='id-scene']")[0]
    # element = driver.find_elements(By.XPATH,"*//div[@aria-label='Street View']")[0]
    # element = driver.find_elements(By.XPATH,"*//canvas[contains(@class, 'widget-scene-canvas')]")[0]
    from selenium.webdriver import ActionChains
    # ActionChains(driver).drag_and_drop_by_offset(element, 100, 0).perform()
    # https://www.selenium.dev/documentation/webdriver/actions_api/mouse/
    # element = driver.find_elements(By.XPATH,"*//div[@id='consent-bump']")[0]
    ActionChains(driver).move_to_element(driver.find_elements(By.XPATH,"*//div[@id='consent-bump']")[0]).perform()

    # print(driver.get_window_size())
    # ActionChains(driver).move_by_offset(0, -300).perform()
    # action = ActionChains(driver)
    # action.click_and_hold(element)
    # action.move_by_offset(0, 20)
    # action.release().perform()
    screenshot = element.screenshot_as_png
    return screenshot, date



def gmap_direct(location):
    location = location.replace(' ','+')
    location = location+',+Singapore'
    uuu = 'https://www.google.com/maps/search/?api=1&query='+location
    # print(uuu)
    driver.get(uuu)
    try: WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "*//button[@id='searchbox-searchbutton']")))
    except:
        driver.find_elements(By.XPATH,"*//form[@action='https://consent.google.com/save']//button")[0].click()
        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "*//button[@id='searchbox-searchbutton']")))
    driver.find_elements(By.XPATH,"*//button[@id='searchbox-searchbutton']")[0].click()
    # if not '!16s' in driver.current_url: return
    try: WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "*//button[@jsaction='pane.heroHeaderImage.click']")))
    except: return
    driver.find_elements(By.XPATH,"*//button[@jsaction='pane.heroHeaderImage.click']")[0].click()
    # print("here")
    try: WebDriverWait(driver, 5).until(lambda driver: ",90t/data=" in driver.current_url)
    except: return
    uu = driver.current_url
    # print(uu)
    fovl = uu.find(',75y,',33)
    pitchl = uu.find(',90t/data',fovl)
    fov, pitch = 80, 95
    url = uu[:fovl+1] + str(fov) + uu[fovl+3:pitchl+1] + str(pitch) + uu[pitchl+3:]
    # print(url)
    driver.get(url)

    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "*//div[@role='contentinfo']/span[7]/span/span/span")))
    element = driver.find_elements(By.XPATH,"*//div[@role='contentinfo']/span[7]/span/span/span")[0]
    mon, yyyy = element.text.split(': ')[1].split(' ')
    months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    mm = months.index(mon)+1
    date= yyyy+'-'+str(mm).zfill(2)
    
    delist=["*//div[@id='titlecard']",
    "*//div[@id='image-header']",
    "*//div[contains(@class, 'scene-footer-container')]",
    "*//div[contains(@class, 'app-viewcard-strip')]",
    "*//div[@id='watermark']",]
    # "*//button[contains(@jsaction, 'drawer.open')]",]
    
    try:
        # driver.find_element_by_xpath("*//button[contains(@jsaction, 'drawer.close')]")
        # print(EC.presence_of_element_located((By.XPATH, "*//button[contains(@jsaction, 'drawer.open')]")))
        # if EC.presence_of_element_located((By.XPATH, "*//button[contains(@jsaction, 'drawer.open')]")):
        driver.find_elements(By.XPATH,"*//button[contains(@jsaction, 'drawer.close')]")[0].click()
        delist.append("*//button[contains(@jsaction, 'drawer.open')]")
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//button[contains(@jsaction, 'drawer.open')]")))
    except: pass

    try:
        driver.find_elements(By.XPATH,"*//div[@role='alert']")[0]
        delist.append("*//div[@role='alert']")
    except: pass

    for delete in delist:
        delement = driver.find_element(By.XPATH, delete)
        driver.execute_script("arguments[0].remove();", delement)
    element = driver.find_elements(By.XPATH,"*//div[@aria-label='Google Maps']")[0]
    from selenium.webdriver import ActionChains
    # print(driver.page_source)
    ActionChains(driver).move_to_element(driver.find_elements(By.XPATH,"*//div[@id='consent-bump']")[0]).perform()
    screenshot = element.screenshot_as_png
    return screenshot, date



def get_screenshot(url):
    driver.get(url)
    WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, "*//div[@role='contentinfo']/span[7]/span/span/span")))
    element = driver.find_elements(By.XPATH,"*//div[@role='contentinfo']/span[7]/span/span/span")[0]
    mon, yyyy = element.text.split(': ')[1].split(' ')
    months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    mm = months.index(mon)+1
    date= yyyy+'-'+str(mm).zfill(2)
    delist=["*//div[@id='titlecard']",
    "*//div[@id='image-header']",
    "*//div[contains(@class, 'scene-footer-container')]",
    "*//div[contains(@class, 'app-viewcard-strip')]",
    "*//div[@id='watermark']",]
    for delete in delist:
        delement = driver.find_element(By.XPATH, delete)
        driver.execute_script("arguments[0].remove();", delement)
    element = driver.find_elements(By.XPATH,"*//div[@aria-label='Street View']")[0]
    screenshot = element.screenshot_as_png
    return screenshot, date


# 40000km, 360deg
# 40000000/360 = 1000000/9 = 111111m/deg


location = "7 THIRD ST" # 
# location = "5 THIRD STREET" # 
# location = "5 Third St" # 
# location = "23 NIM RISE"
# location = "272 MILTONIA CLOSE"
# location = "12 THIAM SIEW AVE" # 7
# location = "740 DUNMAN RD" # https://www.google.com/maps/search/740+DUNMAN+RD+singapore/@1.3091238,103.9007919,3a,75y,322.65h,90t/data=!3m7!1e1!3m5!1s0HN

# fov : ',75y,'
# pitch: ',90t/data'
# olat, olng = gmap_latlng(location)
# print(olat, olng)

# url = heading_url(olat, olng)
# print(url)

# screenshot, date = get_screenshot(url)
# print(date)


screenshot, date = gmap_direct(location)
# print(date)

with open('canvas.png', 'wb') as f:
    f.write(screenshot)

from PIL import Image

# # plt.imshow(np.transpose(pic_content, (1, 2, 0)))
# # image = Image.open(screenshot).convert("RGB")
# image = Image.open('canvas.png').convert("RGB")
# plt.figure(figsize=(5, 3))
# plt.axis('off')
# plt.imshow(image)
# plt.show()


2023-01


In [ ]:
# @title check driver state
# print(driver.current_url)
# # # driver.find_elements(By.XPATH,"*//button[contains(@jsaction, 'drawer.close')]")[0].click()

# element = driver.find_elements(By.XPATH,"*//form[@action='https://consent.google.com/save']//button")[0]
# element.click()
# *//form[@action='https://consent.google.com/save']//button


# element = driver.find_elements(By.XPATH,"*//div[@aria-label='Street View']")[0]
# element = driver.find_elements(By.XPATH,"*//canvas[contains(@class, 'widget-scene-canvas')]")[0]

# screenshot = element.screenshot_as_png
# with open('canvas.png', 'wb') as f:
#     f.write(screenshot)
# driver.save_screenshot("canvas.png")
image = Image.open('canvas.png').convert("RGB")
plt.figure(figsize=(5, 3))
plt.axis('off')
plt.imshow(image)
plt.show()

# print(driver.page_source)


In [82]:
# @title setup pandas
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
df_batch = pd.read_excel("/content/drive/MyDrive/properties_batch.xlsx", engine='openpyxl')
df_all = pd.read_excel("/content/drive/MyDrive/latlat.xlsx", engine='openpyxl')
# display(df)


In [84]:
# df_all = df_all[~df_all['api date'].isnull() &(df_all['bad']!=1)]
# df_all = df_all[df_all['Latitude']=='None']
df_all = df_all[(df_all['bad']==1) & (df_all['blur']!=1)] # 3061
print(len(df_all))
# print(len(df_all[df_all['Latitude']=='None']))
# Property Address Cleaned



3061


In [103]:
print(len(df_all[df_all['name'].isnull()]))
df_all = df_all[df_all['name'].isnull()]


In [ ]:
# import os
# os.mkdir("/content/ggmap_frombad")
# for x in range(1,7):
#     os.mkdir("/content/ggmap_frombad/0"+str(x))


In [ ]:
# @title gmap latlng
import os
from PIL import Image

# [blurred[x] for x in [39,54,62]]

file_dir = '/content/ggmap_frombad'

filename_list = list(df_all['Filename'])
location_list = list(df_all['Property Address Cleaned'])
# latitude_list = list(df_all['Latitude'])
# longitude_list = list(df_all['Longitude'])

# 2454+484
# 1665+484=2149
# 2938
start,end=622,3061 # 3061
t=0
# for img_file, blurrness in zip(img_file_list, blurrness_list):
# for x,filename in enumerate(left):
# for x,(location, filename) in enumerate(zip(location_list, filename_list)):
for x,(location, filename) in enumerate(zip(location_list[start:], filename_list[start:])):
# for x,(olat, olng, filename) in enumerate(zip(latitude_list[start:], longitude_list[start:], filename_list[start:])):
    # if x in [34,40,62]:
    #     continue
    print(x)
    # print(filename)
    if '#' in filename: continue
    # name = os.path.splitext(filename)[0]
    # # img_file=os.path.join(img_dir, filename)
    # name=name.split('/')[-1]
    # # print(name)
    # find_name = name[:-8]
    # print('find_name: ',find_name)

    # olatolng = gmap_latlng(location)
    # # print(olat, olng)
    # if not olatolng: # no meta
    #     continue
    # olat, olng = olatolng

    # print(olat, olng)
    # olat, olng = 1.311398288,103.8964701
    # olat, olng = float(olat), float(olng)
    # url = heading_url(olat, olng)
    # if not url: # no view
    #     continue
    # # print(url)
    # pic_content, date = get_screenshot(url)

    pic_contentdate = gmap_direct(location)
    if not pic_contentdate: # no streetview
        continue
    pic_content, date = pic_contentdate
    print(date)


    try:
        i = df_all.index[df_all['Filename'] == filename][0]
    except: i = df_all.index[df_all['Filename'] == filename.replace('-','/').replace('_','/')][0]
    # print(i)
    # df_all.at[i, 'glat'] = olat
    # df_all.at[i, 'glng'] = olng
    df_all.at[i, 'gdate'] = date
    df_all.at[i, 'name'] = filename
    # print(loc)

    cls=filename.split('. ')[0]
    filename=filename.replace("/", "-")

    pic_dir = file_dir+'/'+cls+'/'+filename + ' ' + date + '.jpg'

    with open(pic_dir, 'wb') as pic_file:            
        pic_file.write(pic_content)
    # image = Image.open(pic_dir).convert("RGB")
    # plt.figure(figsize=(5, 4))
    # plt.axis('off')
    # plt.imshow(image)
    # plt.show()

    # t+=1
    # if t >=5: break

df_all.to_excel('/content/drive/MyDrive/properties_batch.xlsx', index=False, header=False)

# No Street View imagery available here


In [132]:
import os
# allfiles = os.listdir('/content/gmap/01/')
# print(len(allfiles))

count=0
for x in range(1,7):
    allfiles = os.listdir('/content/ggmap_frombad/0'+str(x)+'/')
    count+=len(allfiles)
print(count)


2918


In [28]:
print(list(df_all['Filename']))

['03. 1 GUAN SOON AVE', '04. 1A GUAN SOON AVE', '02. 2 GUAN SOON AVE', '02. 4 GUAN SOON AVE', '05. 6 GUAN SOON AVE', '01. 7 GUAN SOON AVE', '03. 8 GUAN SOON AVE', '02. 9 GUAN SOON AVE', '03. 9A GUAN SOON AVE', '02. 9B GUAN SOON AVE', '03. 10 GUAN SOON AVE', '02. 11 GUAN SOON AVE', '01. 11A GUAN SOON AVE', '03. 12 GUAN SOON AVE', '03. 14 GUAN SOON AVE', '03. 15 GUAN SOON AVE', '03. 15A GUAN SOON AVE', '03. 16 GUAN SOON AVE', '04. 17 GUAN SOON AVE', '04. 17A GUAN SOON AVE', '03. 18 GUAN SOON AVE', '02. 19 GUAN SOON AVE', '02. 19A GUAN SOON AVE', '03. 20 GUAN SOON AVE', '04. 21 GUAN SOON AVE', '03. 22 GUAN SOON AVE', '02. 23 GUAN SOON AVE', '03. 24 GUAN SOON AVE', '03. 25 GUAN SOON AVE', '03. 26 GUAN SOON AVE', '03. 27 GUAN SOON AVE', '03. 28 GUAN SOON AVE', '03. 29 GUAN SOON AVE', '03. 30 GUAN SOON AVE', '03. 31 GUAN SOON AVE', '03. 32 GUAN SOON AVE', '03. 33 GUAN SOON AVE', '03. 34 GUAN SOON AVE', '03. 35 GUAN SOON AVE', '03. 36 GUAN SOON AVE', '03. 37 GUAN SOON AVE', '03. 38 GUAN SOON 

In [68]:
# @title zfill date
img_dir = '/content/ggmap_frombad/06'

t=0
for filename in os.listdir(img_dir):
    # print(filename)
    name = os.path.splitext(filename)[0]

    # print(name)
    date=name.split(' ')[-1]
    # print(date)
    yyyy,m=date.split('-')
    date=yyyy+'-'+m.zfill(2)
    filename1 = ' '.join(name.split(' ')[:-1])+' '+date+'.jpg'
    img_file=os.path.join(img_dir, filename)
    img_file1=os.path.join(img_dir, filename1)
    # print(img_file)
    # print(img_file1)
    os.rename(img_file, img_file1)
    # t+=1
    # if t>4: break


In [97]:
# @title check date / sheet img file
import os
from PIL import Image
import matplotlib.pyplot as plt

img_dir = '/content/ggmap_frombad/06'

t=0
for filename in os.listdir(img_dir):
    # print(filename)
    name = os.path.splitext(filename)[0]
    img_file=os.path.join(img_dir, filename)

    # print(name)
    date=name.split(' ')[-1]
    # print(date)
    find_name = name[:-8]
    # print(find_name)

    try:
        i = df_all.index[df_all['Filename'] == find_name][0]
        # print(i)
    except:
        # print("nope")
        i = df_all.index[df_all['Filename'] == find_name.replace('-','/').replace('_','/')][0]
    # print(i)
    df_all.at[i, 'old'] = date
    # df_all.at[i, 'img file'] = filename
    df_all.at[i, 'name'] = find_name
    # t+=1
    # if t >=5: break

df_all.to_excel('/content/drive/MyDrive/properties_batch.xlsx', index=False, header=False)




In [ ]:
# display(df_all.head())
df_all

In [ ]:
!ls -a /content/ggmap_frombad
# !rm -R /content/gsv/.ipynb_checkpoints
# %cd /content
!ls

!rm -R /content/ggmap_frombad/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/01/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/02/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/03/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/04/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/05/.ipynb_checkpoints
!rm -R /content/ggmap_frombad/06/.ipynb_checkpoints

!zip -r /content/drive/MyDrive/ggmap_frombad3.zip /content/ggmap_frombad


In [ ]:
!unzip /content/drive/MyDrive/ggmap_frombad3.zip -d /


In [56]:
# !rm -R /content/ggmap_frombad

# @title move files
import os
import shutil

# allfiles = os.listdir(source)
# for f in allfiles:
#     src_path = os.path.join(source, f)
#     dst_path = os.path.join(destination, f)
#     shutil.copyfile(src_path, dst_path)

source = '/content/ggmap_frombad/06/06/'
destination = '/content/ggmap_frombad/06'
c=0

for filename in os.listdir(source):
# for src_path in good:
    src_path = os.path.join(source, filename)
    # f=src_path.split('/')[-1]
    # print('/'.split(name))
    # print(destination, f)
    dst_path = os.path.join(destination, filename)
    # print(filename)
    # print(src_path)
    # print(dst_path)
    # shutil.copyfile(src_path, dst_path)
    shutil.move(src_path, dst_path)
    # print(src_path, dst_path)
    # c+=1
    # if c>5: break





In [ ]:
#@title Quit driver
driver.quit()
display.stop_display()